In [24]:
import pandas as pd
import requests
import sys
import time
from IPython.display import display


API_KEY = '02fcb3aacc4b56bc994cb41b31bb9fd30c4e20d342813d2288f86fc6d4420e93'
SCAN_URL = 'https://www.virustotal.com/vtapi/v2/url/scan'
REPORT_URL = 'http://www.virustotal.com/vtapi/v2/url/report'
IN_FILE = 'dns.csv'
SCAN_DATA = {'apikey': API_KEY, 'url': ''}
REPORT_DATA = {'apikey': API_KEY, 'scan': '1', 'resource': ''}
SCAN_OUT_FILE = 'data/scan_ids.csv'
REPORT_OUT_DIR = 'data/reports'


df = pd.read_csv(IN_FILE, encoding = "ISO-8859-1")
url_to_scanid = {}

In [15]:
urls = set(list(df['QNAME'].dropna().unique())) | set(list(df['RRNAME'].dropna().unique()))

In [16]:
display(len(urls))

11226

In [18]:
def chunk(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]

urls = list(urls)
chunked = [l for l in chunk(urls, 4)]
display(chunked[0])
display(chunked[-1])

['www.gstatic.com',
 'www.mozilla.org.cdn.cloudflare.net',
 'str-zedd-production-1686163808.us-east-1.elb.amazonaws.com',
 'instagram.c10r.facebook.com']

['Isi.JhU.Edu', 'segment.prod.bidr.io']

In [19]:
SCAN_DATA['url'] = '\n'.join(chunked[0])
r = requests.post(SCAN_URL, data=SCAN_DATA)
if r.status_code == 200:
    display(r.json())
else:
    display(r.status_code)
    display(r.text)

[{'permalink': 'https://www.virustotal.com/url/5e44ffd45c17a20ad2e1d610128ebb5744a7c4eb10397f75befd2032065a8d09/analysis/1462417590/',
  'resource': 'http://www.gstatic.com/',
  'response_code': 1,
  'scan_date': '2016-05-05 03:06:30',
  'scan_id': '5e44ffd45c17a20ad2e1d610128ebb5744a7c4eb10397f75befd2032065a8d09-1462417590',
  'url': 'http://www.gstatic.com/',
  'verbose_msg': 'Scan request successfully queued, come back later for the report'},
 {'permalink': 'https://www.virustotal.com/url/ede48804dcc79775e340f0c3fcd7d4c147ea76b4037ccad24e2d700ed6f17bd6/analysis/1462417590/',
  'resource': 'http://www.mozilla.org.cdn.cloudflare.net/',
  'response_code': 1,
  'scan_date': '2016-05-05 03:06:30',
  'scan_id': 'ede48804dcc79775e340f0c3fcd7d4c147ea76b4037ccad24e2d700ed6f17bd6-1462417590',
  'url': 'http://www.mozilla.org.cdn.cloudflare.net/',
  'verbose_msg': 'Scan request successfully queued, come back later for the report'},
 {'permalink': 'https://www.virustotal.com/url/e43ebf398d8bedf

In [25]:
failed = []
start = time.time()
count = 0
display('starting...')
with open(SCAN_OUT_FILE, 'a') as f:
    # f.write('url,scan_id\n')
    for chunk in chunked:
        l = set(chunk)
        s = []
        for c in l:
            if c not in url_to_scanid:
                s.append(c)
        if not s:
            continue
        SCAN_DATA['url'] = '\n'.join(s)
        r = requests.post(SCAN_URL, data=SCAN_DATA)
        if r.status_code == 200:
            for d in r.json():
                if d['response_code'] == 1:
                    url_to_scanid[d['url']] = d['scan_id']
                    f.write(d['url'] + ',' + d['scan_id'] + '\n')
        else:
            for u in chunk:
                failed.append(u)
        count += 1
        if count % 10 == 0:
            sys.stdout.write('.')
        time.sleep(18)  # delay it slightly from their limit, just because
end = time.time()
display('Done in {} seconds!'.format(end-start))

'starting...'

........................................................................................................................................................................................................................................................................................

'Done in 52662.532977581024 seconds!'

In [27]:
display(len(failed))

5502

In [28]:
with open('data/failed.txt', 'w') as f:
    f.write('\n'.join(failed))